<a href="https://colab.research.google.com/github/x200706/PythonNotebooks/blob/main/%E8%A8%98%E5%B8%B3%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ctrl+F9執行以下代碼

In [44]:
# pip區
# !pip install streamlit
# !pip install streamlit_option_menu
# !pip install pandas

In [45]:
!mkdir .streamlit

# 掛起雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

mkdir: cannot create directory ‘.streamlit’: File exists


In [6]:
%%writefile .streamlit/config.toml
[theme]

# Primary accent for interactive elements
primaryColor = '#719c4c'

# Background color for the main content area
backgroundColor = '#0E1117'

# Background color for sidebar and most interactive widgets
secondaryBackgroundColor = '#0E1117'

# Color used for almost all text
textColor = '#F9F5F5'

# Font family for all text in the app, except code blocks
# Accepted values (serif | sans serif | monospace)
# Default: "sans serif"
font = "sans serif"

Overwriting .streamlit/config.toml


In [26]:
%%writefile app.py
# streamlit主程式
# 不錯的整理：https://surendraredd.github.io/Books/concepts.html
import streamlit as st
from streamlit_option_menu import option_menu
import pandas as pd
from datetime import datetime

# 站點內容
st.set_page_config(page_title="記帳本", layout="wide")
# 頁面CSS（有些樣式被toml設定檔優先使用，不能從這邊覆蓋）
st.markdown('<style>\
.st-emotion-cache-1dp5vir {\
position: absolute;\
top: 0px;\
right: 0px;\
left: 0px;\
height: 0.125rem;\
background-image: linear-gradient(90deg, #d1ff38, #3cba2f);\
z-index: 999990;\
}\
</style>', unsafe_allow_html=True)

# 側欄內容物陣列
menu_arr = ["記帳管理", "帳戶檢視", "財務BI"]
#TODO記帳管理=新增記帳+記帳總覽 第一版比較陽春 記帳的刪除跟修改請直接改csv謝謝@@ 帳戶也只能用改csv處理
#TODO帳戶跟記帳的關聯是個問題－目前假想 帳戶不能硬刪除 當帳戶被關閉時圖表區不顯示資料 但記帳管理總覽區仍給他關聯（新增時不能使用被關閉的帳戶）

# 生成側欄
with st.sidebar:
    # 側欄分組與主要內容回填
    menu = option_menu("記帳本", menu_arr,
        icons=[],
        menu_icon="cast", default_index=0)

# 兩份csv要存雲端硬碟達到資料持久化
# 分類就放推了 這種寫法關聯太多只是討虐..
def main(): # 這邊應該改成迴圈 枚舉之類的 不然好醜好醜啊
    # 各頁面主要內容
    if menu == "記帳管理":
        st.subheader("💵記帳管理")
        st.info("轉帳或提款請自己記兩筆", icon="⚠️")
        # 讀取雲端硬碟的record.csv
        file_path = '/content/drive/MyDrive/streamlit_acc/record.csv'
        if "df" not in st.session_state:
            st.session_state.df = pd.read_csv(file_path)

        # 讀取雲端硬碟的account.csv
        acc_file_path = '/content/drive/MyDrive/streamlit_acc/account.csv'
        if "acc_df" not in st.session_state:
            st.session_state.acc_df = pd.read_csv(acc_file_path)

        # 新增記帳 檢查帳戶是否存在 異動帳戶
        # 輸入框 太長了做分欄 https://docs.streamlit.io/library/api-reference/layout/st.columns
        col1, col2 = st.columns(2)
        with col1:
          date = st.text_input("日期（YYYYMMDD）", "")
          name = st.text_input("名稱", "")
          io_kind = st.text_input("收支類型（收填I，支填O）", "")

        with col2:
          tag = st.text_input("分類標籤", "")
          price = st.text_input("金額", "")
          account = st.text_input("使用帳戶", "")

        if st.button("新增記帳"):
          # 先檢查使用帳戶是否存在
          #TODO還要檢查是否還能使用
          acc_df = st.session_state.acc_df
          if not acc_df.loc[acc_df['使用帳戶'] == account].empty: # not False
            st.session_state.df = st.session_state.df.append({"日期": date, "名稱": name, "收支類型": io_kind, "分類標籤":tag, "金額":price, "使用帳戶":account}, ignore_index=True)
            st.session_state.df.to_csv(file_path, index=False)

            # 異動帳戶餘額
            # 取回帳戶原來金額
            acc_df['金額'] = pd.to_numeric(acc_df['金額']) # 字串要先轉貨幣
            origin_price = acc_df.loc[acc_df['使用帳戶'] == account, '金額']

            # 根據收支判斷運算方式
            if io_kind == "I":
              acc_df.loc[acc_df['使用帳戶'] == account, '金額'] = origin_price + int(price) # 異動要確實做在原來的csv上才會生效=_=
            else:
              acc_df.loc[acc_df['使用帳戶'] == account, '金額'] = origin_price - int(price)

            acc_df.to_csv(acc_file_path, index=False)

            st.success("新增成功！")
          else:
            st.error("沒有這個戶頭啊")

        df_with_style = st.session_state.df.style.format({"Expense": lambda x : '{:.2f}'.format(x)}) # 其他資料出現None時會導致本設定失效
        # 反正DF自己可以按表頭排序，寫排序導致的問題太多，先不寫了
        st.dataframe(df_with_style, height=500, width=1200)

    if menu == "帳戶檢視":
        st.subheader("帳戶檢視")
        # 讀取雲端硬碟的account.csv
        file_path = '/content/drive/MyDrive/streamlit_acc/account.csv'
        df = pd.read_csv(file_path)
        st.dataframe(df, height=500, width=1200)


    if menu == "財務BI":
        st.subheader("財務BI")



if __name__ == '__main__':
    main()

Overwriting app.py


In [ ]:
# 運行streamlit+臨時隧道 https://discuss.streamlit.io/t/how-to-launch-streamlit-app-from-google-colab-notebook/42399
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.69.0.68



  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.69.0.68:8501

npx: installed 22 in 3.633s
your url is: https://three-pandas-beg.loca.lt
